In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import Dataset

## Important Step:
please change the filename below to the file you want to use for training (This should not include the .csv)

In [2]:
# Specify the training file to take. Change the hashes, filename = '###'
filename = 'MDN_40K_v2'


# Load the CSV file
csv_file = '../3. Cleaned Data/'+filename+'.csv'
df = pd.read_csv(csv_file)

In [3]:
df.head()

,Unnamed: 0,original,corrected
0,10784,He is taller than boys at his age.,He is taller than most boys his age.
1,12111,"All in all ,media help develop society and in...","All in all, media helps develop society and in..."
2,54521,I always tell myself to do hardwork ang try my...,I always tell myself to do hardwork and try my...
3,51688,Waiting for a sunny dateso that we can meet in...,Waiting for a sunny dateso that we can meet on...
4,39393,Taday is the third day .,Today is the third day .


In [4]:
# Change the column names in the dataframe
df.rename(columns = {'corrected_fs':'corrected'}, inplace = True)

In [5]:
# Split the dataset into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert the train and validation DataFrames to Hugging Face's Dataset instances
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [6]:
# Export the validation set
val_file = './../3. Cleaned Data/'+filename+'_val.csv'
val_df.to_csv(val_file, index=False)

In [7]:
# Chose the model
model_name = 'gpt2'

In [8]:
# Assign cuda to the device to use for training
if torch.cuda.is_available(): 
 dev = "cuda:0" 
 print("This model will run on CUDA")
elif  torch.backends.mps.is_available(): 
 dev = "mps:0"
 print("This model will run on MPS")
else:
 dev = "cpu" 
 print("This model will run on CPU")
device = torch.device(dev) 

print(device)

This model will run on CUDA
cuda:0


In [9]:
# Load the tokenizer and the model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
config = GPT2Config.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, config=config).to(device)

In [10]:
# Ensure that the tokenizer uses the same special tokens as GPT-2
tokenizer.pad_token = tokenizer.eos_token

# Function to tokenize and format input-output pairs
def tokenize_function(examples):
    inputs = [f"input: {orig} output: {corr}" for orig, corr in zip(examples["original"], examples["corrected"])]
    return tokenizer(inputs, padding=True, truncation=True, max_length=512, return_tensors='pt')

In [11]:
# Tokenize the train and validation data
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['original', 'corrected'])
val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=['original', 'corrected'])

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [12]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_steps=500,
    save_steps=500,
    warmup_steps=200,
    logging_dir="logs",
    evaluation_strategy="steps",
    logging_steps=100,
)

In [13]:
# Define a custom loss function to focus on the "output" tokens
def custom_loss_function(outputs, labels):
    shift_logits = outputs.logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    loss = torch.nn.CrossEntropyLoss()(shift_logits, shift_labels)
    return loss

In [14]:
# Define a custom Trainer class that inherits from the original Trainer
class CustomTrainer(Trainer):
    
    # Override the compute_loss method to use a custom loss function
    def compute_loss(self, model, inputs, return_outputs=False):
        # Get the labels from the inputs dictionary and remove them from the inputs
        labels = inputs.pop("labels")
        
        # Get the model outputs by passing the inputs to the model
        outputs = model(**inputs)
        
        # Extract the logits from the model outputs
        logits = outputs.logits
        
        # Get the correct dimensions for the shift_labels tensor
        shift_labels = labels[..., 1:].reshape(-1)

        # Reshape the shift_logits tensor to align with the dimensions of the shift_labels tensor
        shift_logits = logits[..., :-1, :].reshape(-1, logits.size(-1))

        loss = torch.nn.CrossEntropyLoss()(shift_logits, shift_labels)

        if return_outputs:
            return loss, outputs
        
        # Otherwise, just return the loss
        return loss

# Create the custom Trainer with the custom loss function
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    
    # Use the DataCollatorForLanguageModeling to handle the data collation
    # Set mlm=False, as we are not using masked language modeling
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

In [15]:
print(model)
print(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)
cuda:0


In [16]:
# Train the model
trainer.train()

/home/timlinux/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/3375 [00:00<?, ?it/s]

{'loss': 2.9702, 'learning_rate': 2.5e-05, 'epoch': 0.09}
{'loss': 2.4512, 'learning_rate': 5e-05, 'epoch': 0.18}
{'loss': 2.4246, 'learning_rate': 4.84251968503937e-05, 'epoch': 0.27}
{'loss': 2.363, 'learning_rate': 4.6850393700787405e-05, 'epoch': 0.36}
{'loss': 2.3563, 'learning_rate': 4.52755905511811e-05, 'epoch': 0.44}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 2.294248580932617, 'eval_runtime': 5.9726, 'eval_samples_per_second': 669.73, 'eval_steps_per_second': 20.929, 'epoch': 0.44}
{'loss': 2.3287, 'learning_rate': 4.370078740157481e-05, 'epoch': 0.53}
{'loss': 2.3128, 'learning_rate': 4.21259842519685e-05, 'epoch': 0.62}
{'loss': 2.3124, 'learning_rate': 4.0551181102362204e-05, 'epoch': 0.71}
{'loss': 2.2699, 'learning_rate': 3.8976377952755905e-05, 'epoch': 0.8}
{'loss': 2.2986, 'learning_rate': 3.740157480314961e-05, 'epoch': 0.89}


...

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 2.210512161254883, 'eval_runtime': 5.9734, 'eval_samples_per_second': 669.637, 'eval_steps_per_second': 20.926, 'epoch': 2.22}
{'loss': 2.0812, 'learning_rate': 1.220472440944882e-05, 'epoch': 2.31}
{'loss': 2.0771, 'learning_rate': 1.062992125984252e-05, 'epoch': 2.4}
{'loss': 2.0745, 'learning_rate': 9.055118110236222e-06, 'epoch': 2.49}
{'loss': 2.083, 'learning_rate': 7.4803149606299226e-06, 'epoch': 2.58}
{'loss': 2.0814, 'learning_rate': 5.905511811023622e-06, 'epoch': 2.67}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 2.2057876586914062, 'eval_runtime': 6.0536, 'eval_samples_per_second': 660.767, 'eval_steps_per_second': 20.649, 'epoch': 2.67}
{'loss': 2.076, 'learning_rate': 4.330708661417323e-06, 'epoch': 2.76}
{'loss': 2.0811, 'learning_rate': 2.755905511811024e-06, 'epoch': 2.84}
{'loss': 2.0915, 'learning_rate': 1.1811023622047244e-06, 'epoch': 2.93}
{'train_runtime': 1183.8578, 'train_samples_per_second': 91.227, 'train_steps_per_second': 2.851, 'train_loss': 2.210198368778935, 'epoch': 3.0}


TrainOutput(global_step=3375, training_loss=2.210198368778935, metrics={'train_runtime': 1183.8578, 'train_samples_per_second': 91.227, 'train_steps_per_second': 2.851, 'train_loss': 2.210198368778935, 'epoch': 3.0})

In [17]:
# Save the trained model and tokenizer
output_dir = "../7. Models/GPT2_"+filename+"/"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Then compress with this command: tar czvf trained_model.tar.gz trained_model/
# Upload to git/drive


('../7. Models/GPT2_MDN_40K_v2/tokenizer_config.json',
 '../7. Models/GPT2_MDN_40K_v2/special_tokens_map.json',
 '../7. Models/GPT2_MDN_40K_v2/vocab.json',
 '../7. Models/GPT2_MDN_40K_v2/merges.txt',
 '../7. Models/GPT2_MDN_40K_v2/added_tokens.json')

In [18]:
# Load trained model
output_dir = "../7. Models/GPT2_"+filename+"/"
model = GPT2LMHeadModel.from_pretrained(output_dir)
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)
